# Overview
My sprint 5 feature is to post the static content of the website on the backend. This feature is a part of the sprint 5 goals. The feature is to be implemented in the backend using flask and flask RESTful. I created 5 files for the 5 different countries. Each file contains the static content of the website for that country. The five countries are : Italian, Chinese, Japanese, Mexican, and Thai. 
The static data is the name of the cuisine, the ingredients, 
instructions, and time it takes to cook the dish. As well as create a function to store recipes to easily open later.

# Backend

Heres an example of the data for a chinese cuisine Kung Pao Chicken:




                        

In [ ]:
"Kung Pao Chicken": {
                "dish": "Kung Pao Chicken",
                "time": 30,
                "ingredients": "Chicken breast (500g), Dried red chilies (10-12)  Peanuts (50g),Soy sauce (2 tbsp),Rice vinegar (1 tbsp),Sugar (1 tsp), Cornstarch (1 tsp), Garlic (3 cloves) Ginger (1-inch piece),Spring onions (2 stalks)",
                "instructions":
                "Cut chicken into small cubes and marinate with soy sauce and cornstarch for 10 minutes. Heat oil in a wok, fry dried chilies and peanuts until fragrant. Add garlic and ginger, stir-fry for 30 seconds. Add chicken and stir-fry until golden brown. Mix soy sauce, rice vinegar, sugar, and stir into the wok. Add spring onions and stir-fry for 2 more minutes before serving.",
            },

These recipes are all seperated by the type of meat or if it is a vegan recipe, and all are stored using a dictionaries and here is an example of one for chicken recipes. 


In [ ]:
def get_chicken_recipe(name):

Then this code will define the recipes and that it will be using a get method to retrieve the recipe data from the API and return it in JSON or give an error message if it was not found. 

In [ ]:
class _KungPaoChicken(Resource):
        def get(self):
            recipe = chinese_recipe_API.get_chicken_recipe("Kung Pao Chicken")
            if recipe:
                return jsonify(recipe)
            return {"Data not found"}, 404

Then here, endpoints for the recipes are created to be called to the frontend. 

In [ ]:
api.add_resource(chinese_recipe_API._KungPaoChicken, '/chinese_recipe/KungPaoChicken')

Heres the code in main.py which creates the flask route to store the recipes variables of name, dish, time, ingredients, and instructions.

In [ ]:
@app.route('/api/chinese_recipe/save_recipe', methods=['POST'])
def save_recipe_route():
    data = request.get_json()
    name = data.get('name')
    dish = data.get('dish')
    time = data.get('time')
    ingredients = data.get('ingredients')
    instructions = data.get('instructions')
    
    recipe = save_recipe(name, dish, time, ingredients, instructions)
    if recipe:
        return jsonify({"message": "Recipe saved successfully", "recipe": recipe.read()}), 201
    else:
        return jsonify({"error": "Recipe could not be created"}), 400

This is another flask route that will use GET method to fetch all the data from the database. This route will be used to fetch all the data when users click on the stored recipes button from the frontend.

In [ ]:
@app.route('/get_recipes', methods=['GET'])
def get_recipes():
    try:
        recipes = Recipe.query.all()  # Fetch all recipes
        recipes_list = [recipe.read() for recipe in recipes]  # Convert to list of dicts
        return jsonify(recipes_list), 200
    except Exception as e:
        return jsonify({"error": str(e)}), 500


This creates the save recipe parameters which are also name, dish, time, ingredients and insructions, then uses a create method in order to save that data and store it into the database.

In [ ]:
def save_recipe(name, dish, time, ingredients, instructions):
    new_recipe = Recipe(name=name, dish=dish, time=time, ingredients=ingredients, instructions=instructions)
    return new_recipe.create()

# Frontend

When users click on Shuffle Recipes, the frontend is setup to fetch 5 recipes from each category and display them on the website using the url endpoints created in the backend. at random along with error handling in case of API errors. 

In [ ]:

const selectedUrls = {
                chicken: apiUrls.chicken[Math.floor(Math.random() * apiUrls.chicken.length)],
                beef: apiUrls.beef[Math.floor(Math.random() * apiUrls.beef.length)],
                vegan: apiUrls.vegan[Math.floor(Math.random() * apiUrls.vegan.length)],
                fish: apiUrls.fish[Math.floor(Math.random() * apiUrls.fish.length)],
                lamb: apiUrls.lamb[Math.floor(Math.random() * apiUrls.lamb.length)]
            };

            try {
                const recipeDataDiv = document.getElementById('recipe-data');
                recipeDataDiv.innerHTML = ''; // Clear previous recipes

                const fetchPromises = Object.values(selectedUrls).map(async (url) => {
                    const response = await fetch(url);
                    if (response.ok) {
                        return await response.json();
                    } else {
                        throw new Error('Error fetching a recipe');
                    }
                });

                const recipes = await Promise.all(fetchPromises);

                recipes.forEach(recipe => {
                    const recipeDiv = document.createElement('div');
                    recipeDiv.classList.add('recipe-card');
                    recipeDiv.innerHTML = `
                        <h3>${recipe.dish}</h3>
                        <p><strong>Time:</strong> ${recipe.time} minutes</p>
                        <p><strong>Ingredients:</strong> ${Array.isArray(recipe.ingredients) ? recipe.ingredients.join(', ') : recipe.ingredients}</p>
                        <p><strong>Instructions:</strong> ${recipe.instructions}</p>
                        <button onclick='saveRecipe(${JSON.stringify(recipe)})'>Save Recipe</button>
                    `;
                    recipeDataDiv.appendChild(recipeDiv);
                });

                recipeDataDiv.style.display = 'flex';  
            } catch (error) {
                document.getElementById('recipe-data').innerText = `Error: ${error.message}`;
            }
        }

Users will also be able to store recipes they catch their eyes but still want to keep shuffling. This will be done using a button with the following code for this function which will cause the recipe to be stored in the database.

In [ ]:
async function saveRecipe(recipe) {
        try {
            const response = await fetch('http://127.0.0.1:8887/save_recipe', {
                method: 'POST',
                headers: {
                    'Content-Type': 'application/json'
                },
                body: JSON.stringify({
                    "title": recipe.dish,
                    "ingredients": recipe.ingredients,
                    "instructions": recipe.instructions,
                    "time": recipe.time
                })
            });


In order to view the recipes that users store they can click a button below the shuffle recipes button called Stored recipes, These will be called from the database with the following code.

In [ ]:
async function viewStoredRecipes() {
        try {
            const response = await fetch('http://127.0.0.1:8887/get_recipes');
            const contentType = response.headers.get("content-type");

            if (contentType && contentType.indexOf("application/json") !== -1) {
                const recipes = await response.json();

                const recipeDataDiv = document.getElementById('recipe-data');
                recipeDataDiv.innerHTML = ''; // Clear previous recipes

                recipes.forEach(recipe => {
                    const recipeDiv = document.createElement('div');
                    recipeDiv.classList.add('recipe-card');
                    recipeDiv.innerHTML = `
                        <h3>${recipe.title}</h3>
                        <p><strong>Ingredients:</strong> ${recipe.ingredients}</p>
                        <p><strong>Instructions:</strong> ${recipe.instructions}</p>
                    `;
                    recipeDataDiv.appendChild(recipeDiv);
                });
            } else {
                throw new Error('Invalid response from server');
            }